Import


In [1]:
from itertools import combinations
import pandas as pd
import itertools
import copy
import uuid

Data

In [2]:
df = pd.read_csv('BankChurners_real.csv')

In [3]:
df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Zipcode
0,768805383,Existing Customer,45,M,3,High School,Married,61501,Blue,39,5,1,3,12691.0,777,11914.0,18914
1,818770008,Existing Customer,49,F,5,Graduate,Single,21723,Blue,44,6,1,2,8256.0,864,7392.0,18625
2,713982108,Existing Customer,51,M,3,Graduate,Married,118804,Blue,36,4,1,0,3418.0,0,3418.0,16878
3,769911858,Existing Customer,40,F,4,High School,Unknown,35130,Blue,34,3,4,1,3313.0,2517,796.0,17490
4,709106358,Existing Customer,40,M,3,Uneducated,Married,62303,Blue,21,5,1,0,4716.0,0,4716.0,17879


Definition

In [4]:
df.columns

Index(['CLIENTNUM', 'Attrition_Flag', 'Customer_Age', 'Gender',
       'Dependent_count', 'Education_Level', 'Marital_Status', 'Income',
       'Card_Category', 'Months_on_book', 'Total_Relationship_Count',
       'Months_Inactive_12_mon', 'Contacts_Count_12_mon', 'Credit_Limit',
       'Total_Revolving_Bal', 'Avg_Open_To_Buy', 'Zipcode'],
      dtype='object')

In [5]:
Q = ['Gender', 'Customer_Age', 'Income', 'Zipcode' ]

Generalization Hirarchy

Gender:

    m, f -> p

Customer Age:

    XX -> 0 - 20, 20 - 30, 30 - 40, 40 - 50, 60 +

Income:

    XXXXX -> 0 - 40k, 40k - 60k, 60k - 80k, 80k - 100k, 100k - 120k, 120k +


Zipcode:

    XXXXX
    XXXX*
    XXX**
    XX***

In [6]:
class NoGeneralization(Exception):
  pass

# class NotImplemented(Exception)

class ValueGeneralization:

  def __init__(self, level) :
    self.level = level

  def generalize(self, value):
    raise NotImplementedError

  def __copy__(self):
    result = self.__class__(self.level)
    result.__dict__.update(self.__dict__)

    return result

  def __deepcopy__(self, memo=None):
    result = self.__class__(self.level)
    result.__dict__ = copy.deepcopy(self.__dict__)
    return result

  def __str__(self):
    attributes = [f"{key}={getattr(self, key)}" for key in self.__dict__]
    return f"Node({', '.join(attributes)})"

class Gender(ValueGeneralization):

  def generalize(self, value):
    if self.level == 0:
      return value
    elif self.level == 1:
      return 'p'
    else:
      raise NoGeneralization("The level is not defined")

class Customer_Age(ValueGeneralization):

  def generalize(self, value):
    if self.level == 0:
      return value
    elif self.level == 1:
      return self.first_level_generalize(value)
    else:
      raise NoGeneralization("The level is not defined")

  def first_level_generalize(self, value):
    if int(value) >= 0 and int(value) < 20:
      return '0 - 20'
    if int(value) >= 20 and int(value) < 30:
      return '20 - 30'
    if int(value) >= 30 and int(value) < 40:
      return '30 - 40'
    if int(value) >= 40 and int(value) < 50:
      return '40 - 50'
    if int(value) >= 60:
      return '60 + '

class Income(ValueGeneralization):

  def generalize(self, value):
    if self.level == 0:
      return value
    elif self.level == 1:
      return self.first_level_generalize(value)
    else:
      raise NoGeneralization("The level is not defined")

  def first_level_generalize(self, value):
    if int(value) >= 0 and int(value) < 40000:
      return '0 - 40k'
    if int(value) >= 40000 and int(value) < 60000:
      return '40k - 60k'
    if int(value) >= 60000 and int(value) < 80000:
      return '60k - 80k'
    if int(value) >= 80000 and int(value) < 100000:
      return '80k - 100k'
    if int(value) >= 100000 and int(value) < 120000:
      return '100k - 120k'
    if int(value) >= 120000:
      return '120k + '

class Zipcode(ValueGeneralization):

  def generalize(self, value):
    if self.level == 0:
      return value
    elif self.level == 1:
      return self.first_level_generalize(value)
    elif self.level == 2:
      return self.second_level_generalize(value)
    elif self.level == 3:
      return self.third_level_generalize(value)
    else:
      raise NoGeneralization("The level is not defined")

  def first_level_generalize(self, value):
    return str(value)[:-1] + '*'

  def second_level_generalize(self, value):
    return str(value)[:-2] + '**'

  def third_level_generalize(self, value):
    return str(value)[:-3] + '***'

# Q = ['Gender', 'Customer_Age', 'Income', 'Zipcode' ]
generalization_combination = {'Gender' : 1, 'Customer_Age' : 1, 'Income' : 1, 'Zipcode' : 3}

Frequency set

In [7]:
def get_frequenct_set(df, columns):
  return df.groupby(columns).size()

In [8]:
def is_K_Anonymous(df, q, k):
  frequency = get_frequenct_set(df, q)
  for fr in frequency:
    if fr < k:
      return False
  return True

In [9]:
get_frequenct_set(df, Q)

Gender  Customer_Age  Income  Zipcode
F       26            1224    16352      1
                      3496    17355      1
                      7962    17199      1
                      8872    16415      1
                      11564   16031      1
                                        ..
M       67            56551   17829      1
        68            32582   17111      1
                      87406   16981      1
        70            14653   16606      1
        73            55726   16091      1
Length: 10127, dtype: int64

Define Nodes

In [10]:
class Node:
  def __init__(self):
    self.id = uuid.uuid4()
    self.anonymized = False
    self.marked = False
    self.direct_nodes = []
    self.height = 0

  def __copy__(self):
    result = Node()
    result.__dict__ = copy.copy(self.__dict__)
    result.id = uuid.uuid4()
    return result

  def __deepcopy__(self, memo=None):
    result = Node()
    result.__dict__= copy.deepcopy(self.__dict__)
    result.id = uuid.uuid4()
    return result

  def __str__(self):
    attributes = [f"{key}={str(getattr(self, key))}" for key in self.__dict__]
    return f"Node({', '.join(attributes)})"


Create hirarchy

In [11]:
def is_direct_generalization(n1, n2, columns):
  one_diff = False
  for column in columns:
    level_diff = getattr(n2, column).level - getattr(n1, column).level
    if level_diff == 1 and not one_diff:
      one_diff = True
    elif level_diff != 0:
      return False

  return True

In [12]:
def add_direct_generalization(nodes, columns):
  for n1 in nodes:
    n1.direct_nodes = []
    for n2 in nodes:
      if n1.id == n2.id:
        continue
      if is_direct_generalization(n1, n2, columns):
        n1.direct_nodes.append(n2.id)

  return nodes

In [13]:

def create_hirarchy(generalization_combination, columns):
  result = []
  node = None
  height = 0
  indexes = [list(range(generalization_combination[key] +1 )) for key in columns]
  combinations = list(itertools.product(*indexes))
  for combination in combinations:
    node = Node()
    for i in range(len(columns)):
      setattr(node, columns[i], globals()[columns[i]](combination[i]))

    result.append(node)

  return add_direct_generalization(result, columns)


In [14]:
nodes = create_hirarchy(generalization_combination, ['Gender', 'Zipcode'])
for num in nodes:
  print(num)

print(len(nodes))

Node(id=d2b5e6b3-edcf-4444-9b8a-7e620ab829b5, anonymized=False, marked=False, direct_nodes=[UUID('f72a2fe2-2641-421c-b16a-7a1fb7245189'), UUID('8e8ee018-6aa5-4ec0-beb8-600f26a288e4')], height=0, Gender=Node(level=0), Zipcode=Node(level=0))
Node(id=f72a2fe2-2641-421c-b16a-7a1fb7245189, anonymized=False, marked=False, direct_nodes=[UUID('a3fe6c37-457b-4488-89b0-12db27d71df5'), UUID('59e370e4-1c0d-4759-8e74-8fc895f7bf86')], height=0, Gender=Node(level=0), Zipcode=Node(level=1))
Node(id=a3fe6c37-457b-4488-89b0-12db27d71df5, anonymized=False, marked=False, direct_nodes=[UUID('669ac2fa-c778-4375-aee2-f6dfc76103a1'), UUID('f55cef33-10c8-4d8d-9d4f-593d3f108241')], height=0, Gender=Node(level=0), Zipcode=Node(level=2))
Node(id=669ac2fa-c778-4375-aee2-f6dfc76103a1, anonymized=False, marked=False, direct_nodes=[UUID('82fe8768-c8fd-4ff2-8fdd-af9e9c3dc809')], height=0, Gender=Node(level=0), Zipcode=Node(level=3))
Node(id=8e8ee018-6aa5-4ec0-beb8-600f26a288e4, anonymized=False, marked=False, direct_n

In [15]:
def generalize_row(row, node, columns):
  for column in columns:
    row[column] = getattr(node, column).generalize(row[column])

  return row
    # print(row)


In [16]:
df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Zipcode
0,768805383,Existing Customer,45,M,3,High School,Married,61501,Blue,39,5,1,3,12691.0,777,11914.0,18914
1,818770008,Existing Customer,49,F,5,Graduate,Single,21723,Blue,44,6,1,2,8256.0,864,7392.0,18625
2,713982108,Existing Customer,51,M,3,Graduate,Married,118804,Blue,36,4,1,0,3418.0,0,3418.0,16878
3,769911858,Existing Customer,40,F,4,High School,Unknown,35130,Blue,34,3,4,1,3313.0,2517,796.0,17490
4,709106358,Existing Customer,40,M,3,Uneducated,Married,62303,Blue,21,5,1,0,4716.0,0,4716.0,17879


In [17]:
def mark_direc_generalization(nodes, node):
  for n in nodes:
    if n.id in node.direct_nodes:
      n.is_marked = True
  return nodes

In [18]:
# Q = ['Gender', 'Customer_Age', 'Income', 'Zipcode' ]
def incognito(df, k, Q):
  n = len(Q)
  nodes = []
  S = [[] for _ in range(n)]
  columns  = []
  for i in range(1, len(Q) + 1):
    column_combination = [list(comb) for comb in combinations(Q, i)]
    # for each size of possible Q for example at first try to one, two and so on untile size of attributes
    for columns in column_combination:
      nodes = create_hirarchy(generalization_combination, columns)
      for node in nodes:
        if not node.marked:
          node.marked = True
          new_df = df.apply(generalize_row, args=(node,columns), axis=1)
          if is_K_Anonymous(new_df, columns, k):
            node.anonymized = True
            S[i].append(node)
            for n in nodes:
              if n.id in node.direct_nodes:
                n.is_marked = True
                S[i].append(n)


  return S



In [19]:
s = incognito(df, 2, Q)

In [20]:
for _ in s:
  for d in _:
    print(d)

Node(id=895d68d0-cdef-4212-b275-feb87b0d7bcf, anonymized=True, marked=True, direct_nodes=[UUID('1b635560-9ae2-4de0-92ea-e60b83ab36c9')], height=0, Gender=Node(level=0))
Node(id=1b635560-9ae2-4de0-92ea-e60b83ab36c9, anonymized=True, marked=True, direct_nodes=[], height=0, Gender=Node(level=1), is_marked=True)
Node(id=1b635560-9ae2-4de0-92ea-e60b83ab36c9, anonymized=True, marked=True, direct_nodes=[], height=0, Gender=Node(level=1), is_marked=True)
Node(id=2ed07649-3cd6-4555-b21d-493ff255b8b1, anonymized=True, marked=True, direct_nodes=[], height=0, Customer_Age=Node(level=1))
Node(id=0248dd9f-660c-4905-be68-91635f14c71d, anonymized=True, marked=True, direct_nodes=[], height=0, Income=Node(level=1))
Node(id=6ddc9adc-b3c3-4ca9-aac0-4ece993d0ba1, anonymized=True, marked=True, direct_nodes=[UUID('1e6d3a1c-2f9b-405f-83df-698056a296c9')], height=0, Zipcode=Node(level=1))
Node(id=1e6d3a1c-2f9b-405f-83df-698056a296c9, anonymized=True, marked=True, direct_nodes=[UUID('1d6e0884-667b-47f2-8ecd-7d8